# Creating an Availability Group (AG) Using DBATools
## Prerequisites
* At least two servers joined to a Windows Server Failover Cluster without shared storage
* Each server must be running Enterprise or Developer Edition and must be the same version
* The log and data paths should be the same
* Enable Always On Availability Groups for each instance to be added - NOTE: You can enable Always On Availability Groups from SQL Server Configuration Manager or by using the DBATools cmdlet ```
Enable-DbaAgHadr.
```
* Each database to be added to the Availability Group must be in full recovery and have a full backup run

This example uses Automatic Seeding, which is the easiest way to set up an AG.  You can find information about the other seeding options [here](https://docs.microsoft.com/en-us/sql/database-engine/availability-groups/windows/use-the-availability-group-wizard-sql-server-management-studio?view=sql-server-ver15).

## Building an AG Using DBATools

Once the prerequisites are met, ```New-DbaAvailabilityGroup``` builds your AG with a single command.




In [19]:
$params = @{
 Primary = 'fbglexhavm2'
 Secondary = 'fbglexhavm3'
 Name = 'fbglexhaag1'
 Database= 'WWI'
 SeedingMode = 'Automatic'
 DatabaseHealthTrigger = $true
 DtcSupport = $true
 Confirm = $false
 }

New-DbaAvailabilityGroup @params;

. {


>> $params = @{
>>  Primary = 'fbglexhavm2'
>>  Secondary = 'fbglexhavm3'
>>  Name = 'fbglexhaag1'
>>  Database= 'WWI'
>>  SeedingMode = 'Automatic'
>>  DatabaseHealthTrigger = $true
>>  DtcSupport = $true
>>  Confirm = $false
>>  }
>> 
>> New-DbaAvailabilityGroup @params;
>> }
>> 




ComputerName               : fbglexhavm2
InstanceName               : MSSQLSERVER
SqlInstance                : fbglexhavm2
LocalReplicaRole           : Primary
AvailabilityGroup          : fbglexhaag1
PrimaryReplica             : fbglexhavm2
ClusterType                : Wsfc
DtcSupportEnabled          : True
AutomatedBackupPreference  : Secondary
AvailabilityReplicas       : {fbglexhavm2, fbglexhavm3}
AvailabilityDatabases      : {WWI}
AvailabilityGroupListeners : {}





# Create Agent Jobs to Test Sync-DbaAvailabilityGroup

One of the challenges of running an AG is keeping non-database objects like logins and jobs synced between the replicas.  DbaTools provides cmdlet ```Sync-DbaAvailabilityGroup``` to maintain replicas.  To test this, I will use ```Install-DbaMaintenanceSolution``` to install Ola Hallengren's Maintenance Solution jobs.

In [20]:
Install-DbaMaintenanceSolution -SqlInstance fbglexhavm2 -Database master -ReplaceExisting -InstallJobs;


ComputerName InstanceName SqlInstance Results


------------ ------------ ----------- -------
fbglexhavm2  MSSQLSERVER  fbglexhavm2 Success




Confirm the jobs have been created using `Get-DbaAgentJob`. The jobs in the Maintenance Solution are all in category Database Maintenance.  Also, check the job list on the secondary replica.

In [21]:
Get-DbaAgentJob -SqlInstance fbglexhavm2 -Category 'Database Maintenance' | Select ComputerName, Name;
Get-DbaAgentJob -SqlInstance fbglexhavm3 -Category 'Database Maintenance' | Select ComputerName, Name;

. {


>> Get-DbaAgentJob -SqlInstance fbglexhavm2 -Category 'Database Maintenance' | Select ComputerName, Name;
>> Get-DbaAgentJob -SqlInstance fbglexhavm3 -Category 'Database Maintenance' | Select ComputerName, Name;
>> }
>> 


ComputerName Name                                     
------------ ----                                     
fbglexhavm2  CommandLog Cleanup                       
fbglexhavm2  DatabaseBackup - SYSTEM_DATABASES - FULL 
fbglexhavm2  DatabaseBackup - USER_DATABASES - DIFF   
fbglexhavm2  DatabaseBackup - USER_DATABASES - FULL   
fbglexhavm2  DatabaseBackup - USER_DATABASES - LOG    
fbglexhavm2  DatabaseIntegrityCheck - SYSTEM_DATABASES
fbglexhavm2  DatabaseIntegrityCheck - USER_DATABASES  
fbglexhavm2  IndexOptimize - USER_DATABASES           
fbglexhavm2  Output File Cleanup                      
fbglexhavm2  sp_delete_backuphistory                  
fbglexhavm2  sp_purge_jobhistory                      
fbglexhavm3  CommandLog Cleanup                       
fbglexhavm3  Output File Cleanup                      
fbglexhavm3  sp_delete_backuphistory                  
fbglexhavm3  sp_purge_jobhistory                      




Once the jobs have been created, sync them to the secondary replica using ```Sync-DbaAvailabilityGroup```. The cmdlet will sync the following object types:
* SpConfigure 
* CustomErrors 
* Credentials 
* DatabaseMail 
* LinkedServers 
* Logins 
* LoginPermissions 
* SystemTriggers 
* DatabaseOwner 
* AgentCategory 
* AgentOperator 
* AgentAlert 
* AgentProxy 
* AgentSchedule 
* AgentJob

For this example, everything but Agent jobs will be excluded.   

In [22]:
$primary = 'fbglexhavm2';
$secondary = 'fbglexhavm3';
$ag = 'fbglexhaag1';
$exclude = "SpConfigure","CustomErrors","Credentials","DatabaseMail","LinkedServers","Logins","LoginPermissions","SystemTriggers","DatabaseOwner","AgentCategory","AgentOperator","AgentAlert","AgentProxy","AgentSchedule";

Sync-DbaAvailabilityGroup -Primary $primary -Secondary $secondary -AvailabilityGroup $ag -exclude $exclude;

Type      Name                                      Status     Notes                        
----      ----                                      ------     -----                        
Agent Job CommandLog Cleanup                        Skipped    Already exists on destination
Agent Job DatabaseBackup - SYSTEM_DATABASES - FULL  Successful                              
Agent Job DatabaseBackup - USER_DATABASES - DIFF    Successful                              
Agent Job DatabaseBackup - USER_DATABASES - FULL    Successful                              
Agent Job DatabaseBackup - USER_DATABASES - LOG     Successful                              
Agent Job DatabaseIntegrityCheck - SYSTEM_DATABASES Successful                              
Agent Job DatabaseIntegrityCheck - USER_DATABASES   Successful                              
Agent Job IndexOptimize - USER_DATABASES            Successful                              
Agent Job Output File Cleanup                       Skipped    Already

# Failover Availability Group

Once the secondary replica has been synced, the AG is ready to be failed over. The ```Invoke-DbaAgFailover``` cmdlet will perform the failover.

In [29]:
$newprimary = 'fbglexhavm2';
$oldprimary = 'fbglexhavm3'
$ag = 'fbglexhaag1';
Invoke-DbaAgFailover -SqlInstance $newprimary -AvailabilityGroup $ag -Force;
Resume-DbaAgDbDataMovement -SqlInstance $oldprimary -AvailabilityGroup $ag -Database WWI -Confirm:$false;



ComputerName               : fbglexhavm2
InstanceName               : MSSQLSERVER
SqlInstance                : fbglexhavm2
LocalReplicaRole           : Primary
AvailabilityGroup          : fbglexhaag1
PrimaryReplica             : fbglexhavm2
ClusterType                : Wsfc
DtcSupportEnabled          : True
AutomatedBackupPreference  : Secondary
AvailabilityReplicas       : {fbglexhavm2, fbglexhavm3}
AvailabilityDatabases      : {WWI}
AvailabilityGroupListeners : {}



ComputerName         : fbglexhavm3
InstanceName         : MSSQLSERVER
SqlInstance          : fbglexhavm3
AvailabilityGroup    : fbglexhaag1
LocalReplicaRole     : Secondary
Name                 : WWI
SynchronizationState : NotSynchronizing
IsFailoverReady      : False
IsJoined             : True
IsSuspended          : False



